# Generating Useful Wikidata Files

This notebook generates files that contain derived data that is useful in many applications. The input to the notebook is the full Wikidata or a subset of Wikidata. It also works for arbutrary KGs as long as they follow the representation requirements of Wikidata:

- the *instance of* relation is represented using the `P31` property
- the *subclass of* relation is represented using the `P279` property
- all properties declare a datatype, and the data types must be one of the datatypes in Wikidata.

Inputs:

- `claims_file`: contains all statements, which consist of edges `node1/label/node2` where `label` is a property in Wikidata (e.g., sitelinks, labels, aliases and description are not in the claims file.
- `item_file`: the subset of the `claims_file` consistin of edges for property of data type `wikibase-item`
- `label_file`, `alias_file` and `description_file` containing labels, aliases and descriptions. It is assume that these files contain the labels, aliases and descriptions of all nodes appearing in the claims file. Users may provide these files for specific languages only.

Outputs:

- **Instance of (P31):** `derived.P31.tsv.gz` contains all the `instance of (P31)` edges present in the claims file.
- **Subclass of (P279):** `derived.P279.tsv.gz` contains all the `subclass of (P279)` edges present in the claims file.
- **Is A (isa):** `derived.isa.tsv.gz` contains edges `node`isa/node2` where either `node1/P31/node2` or `node1/P279/node2`
- **Closure of subclass of (P279star):** `derived.P279star.tsv.gz` contains edges `node1/P279star/node2` where `node2` is reachable from `node1` via zero or more hops using the `P279` property. Note that for example, `Q44/P279star/Q44`. An example when this file is useful is when you want to find all the instance of a class, including instances of subclasses of the given class.
- **In/out degrees:** `metadata.out_degree.tsv.gz` contains the out degree of every node, and `metadata.in_degree.tsv.gz` contains the in degree of every node.
- **Pagerank:** outputs page rank on the directed graph in `metadata.pagerank.directed.tsv.gz` and page rank of the directed graph in `metadata.pagerank.undirected.tsv.gz`.

### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p claims_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.wikibase-item.tsv.gz \
-p property_item_file = /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v4/part.property.wikibase-item.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no 
-p languages es,ru,zh-cn
```

In [1]:
# Parameters

# Folder on local machine where to create the output and temporary folders
output_path = "/data/amandeep/wikidata-20210215"

# The names of the output and temporary folders
output_folder = "useful_wikidata_files"
temp_folder = "temp.useful_wikidata_files"

# The location of input files
wiki_root_folder = "/data/amandeep/wikidata-20210215/"
claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"

label_all = "labels.tsv.gz"
alias_all = "aliases.tsv.gz"
description_all = "descriptions.tsv.gz"

# Location of the cache database for kypher
cache_path = "/data/amandeep/wikidata-20210215/temp.useful_wikidata_files"

# Whether to delete the cache database
delete_database = False

# Whether to compute pagerank as it may not run on the laptop
compute_pagerank = True
languages = 'ru,es,zh-cn,de,it,nl,pl,fr,pt,sv'

In [2]:
# Parameters
output_path = "/nas/home/hrathod/output/for_novartis/novartis"
output_folder = "useful_files"
temp_folder = "temp.useful_files"
wiki_root_folder = "/nas/home/hrathod/output/for_novartis/novartis/parts/"
cache_path = "/nas/home/hrathod/output/for_novartis/novartis/temp.useful_files"
languages = "en"
compute_pagerank = True
delete_database = False


In [3]:
languages = languages.split(',')

In [4]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

## Set up environment and folders to store the files

- `OUT` folder where the output files go
- `TEMP` folder to keep temporary files , including the database
- `kgtk` shortcut to invoke the kgtk software
- `kypher` shortcut to invoke `kgtk query with the cache database
- `CLAIMS` the `all.tsv` file of wikidata that contains all edges except label/alias/description
- `LABELS` the file with the English labels
- `ITEMS` the wikibase-item file (currently does not include node1 that are properties so for now we need the net file
- `STORE` location of the cache file

In [5]:
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
os.environ['OUT'] = "{}/{}".format(output_path, output_folder)
os.environ['TEMP'] = "{}/{}".format(output_path, temp_folder)
os.environ['kgtk'] = "kgtk"
os.environ['kgtk'] = "kgtk --debug"
os.environ['kypher'] = "kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['CLAIMS'] = wiki_root_folder + claims_file
os.environ['LABELS'] = wiki_root_folder + label_file
os.environ['ALIASES'] = wiki_root_folder + alias_file
os.environ['DESCRIPTIONS'] = wiki_root_folder + description_file
os.environ['ITEMS'] = wiki_root_folder + item_file

Echo the variables to see if they are all set correctly

In [6]:
!echo $OUT
!echo $TEMP
!echo $kgtk
!echo $kypher
!echo $CLAIMS
!echo $LABELS
!echo $ALIASES
!echo $LABELS
!echo $DESCRIPTIONS
!echo $STORE
!alias col="column -t -s $'\t' "

/nas/home/hrathod/output/for_novartis/novartis/useful_files
/nas/home/hrathod/output/for_novartis/novartis/temp.useful_files
kgtk --debug
kgtk --debug query --graph-cache /nas/home/hrathod/output/for_novartis/novartis/temp.useful_files/wikidata.sqlite3.db
/nas/home/hrathod/output/for_novartis/novartis/parts/claims.tsv.gz
/nas/home/hrathod/output/for_novartis/novartis/parts/labels.en.tsv.gz
/nas/home/hrathod/output/for_novartis/novartis/parts/aliases.en.tsv.gz
/nas/home/hrathod/output/for_novartis/novartis/parts/labels.en.tsv.gz
/nas/home/hrathod/output/for_novartis/novartis/parts/descriptions.en.tsv.gz
/nas/home/hrathod/output/for_novartis/novartis/temp.useful_files/wikidata.sqlite3.db


Go to the output directory and create the subfolders for the output files and the temporary files

In [7]:
cd $output_path

/nas/home/hrathod/output/for_novartis/novartis


In [8]:
!mkdir -p $OUT
!mkdir -p $TEMP

Clean up the output and temp folders before we start

In [9]:
# !rm $OUT/*.tsv $OUT/*.tsv.gz
# !rm $TEMP/*.tsv $TEMP/*.tsv.gz

In [10]:
if delete_database:
    print("Deleteddatabase") 
    !rm $STORE

In [8]:
!ls -l $OUT
!ls $TEMP
!ls -l "$CLAIMS"
!ls -l "$LABELS"
!ls -l "$ALIASES"
!ls -l "$LABELS"
!ls -l "$DESCRIPTIONS"
!ls $STORE

total 14773863
-rw-r--r-- 1 hrathod div22   89935660 Apr  8 13:47 aliases.en.tsv.gz
-rw-r--r-- 1 hrathod div22 7760608762 Apr  9 01:57 derived.isastar.tsv.gz
-rw-r--r-- 1 hrathod div22  117261297 Apr  8 16:54 derived.isa.tsv.gz
-rw-r--r-- 1 hrathod div22  500270514 Apr  8 16:06 derived.P279star.tsv.gz
-rw-r--r-- 1 hrathod div22   37632946 Apr  8 13:59 derived.P279.tsv.gz
-rw-r--r-- 1 hrathod div22 7833861139 Apr  9 19:37 derived.P31P279star.tsv.gz
-rw-r--r-- 1 hrathod div22  429838311 Apr  8 13:56 derived.P31.tsv.gz
-rw-r--r-- 1 hrathod div22  174388582 Apr  8 13:50 descriptions.en.tsv.gz
-rw-r--r-- 1 hrathod div22  389162815 Apr  8 13:46 labels.en.tsv.gz
-rw-r--r-- 1 hrathod div22   37158445 Apr 10 07:28 metadata.in_degree.tsv.gz
-rw-r--r-- 1 hrathod div22  229821867 Apr 10 01:58 metadata.out_degree.tsv.gz
-rw-r--r-- 1 hrathod div22  882646108 Apr  9 23:09 metadata.pagerank.directed.tsv.gz
-rw-r--r-- 1 hrathod div22  984042006 Apr 10 01:23 metadata.pagerank.undirected.tsv.gz
-rw-r--r-

In [12]:
!zcat < "$CLAIMS" | head | col

id	node1	label	node2	node2;wikidatatype	rank
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video" url	normal
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	url	normal
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	wikibase-item	normal
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	wikibase-property	normal
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	wikibase-property	normal

gzip: P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	wikibase-property	normal
stdout: Broken pipe
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	wikibase-property	normal
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950	wikibase-item	normal
P10-P1855-Q4504-a69d2c73-0	P10	P1855	Q4504	wikibase-item	normal


### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [13]:
!$kypher -i "$CLAIMS" --limit 10 | col 

[2021-04-08 13:38:26 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	node2;wikidatatype	rank
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video" url	normal
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	url	normal
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	wikibase-item	normal
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	wikibase-property	normal
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	wikibase-property	normal
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	wikibase-property	normal
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	wikibase-property	normal
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950	wikibase-item	normal
P10-P1855-Q4504-a69d2c73-0	P10	P1855	Q4504	wikibase-item	normal
P10-P1855-Q69063653-c8cdb04c-0	P10	P1855	Q69063653	wikibase-item	normal


Force creation of the index on the label column

In [14]:
!$kypher -i "$CLAIMS" -o - \
--match '(i)-[:P31]->(c)' \
--limit 5 \
| column -t -s $'\t' 

[2021-04-08 13:38:29 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     LIMIT ?
  PARAS: ['P31', 5]
---------------------------------------------
id                              node1  label  node2      node2;wikidatatype  rank
P10-P31-Q18610173-85ef4d24-0    P10    P31    Q18610173  wikibase-item       normal
P1000-P31-Q18608871-093affb5-0  P1000  P31    Q18608871  wikibase-item       normal
P1001-P31-Q15720608-deeedec9-0  P1001  P31    Q15720608  wikibase-item       normal
P1001-P31-Q22984026-8beb0cfe-0  P1001  P31    Q22984026  wikibase-item       normal
P1001-P31-Q22997934-1e5b1a96-0  P1001  P31    Q22997934  wikibase-item       normal


Force creation of the index on the node2 column

In [15]:
!$kypher -i "$CLAIMS" -o - \
--match '(i)-[r]->(:Q5)' \
--limit 5 \
| column -t -s $'\t' 

[2021-04-08 13:38:31 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."node2"=?
     LIMIT ?
  PARAS: ['Q5', 5]
---------------------------------------------
id                         node1  label  node2  node2;wikidatatype  rank
P1424-P1855-Q5-47bdcd17-0  P1424  P1855  Q5     wikibase-item       normal
P1552-P1855-Q5-53b667e4-0  P1552  P1855  Q5     wikibase-item       normal
P1963-P1855-Q5-1ba43aca-0  P1963  P1855  Q5     wikibase-item       normal
P3055-P1629-Q5-fb63cfeb-0  P3055  P1629  Q5     wikibase-item       normal
P5869-P1855-Q5-3a19317f-0  P5869  P1855  Q5     wikibase-item       normal


### Count the number of edges

Counting takes a long time

In [16]:
!$kypher -i "$CLAIMS" \
--match '()-[r]->()' \
--return 'count(r) as count' \
--limit 10

[2021-04-08 13:38:35 query]: SQL Translation:
---------------------------------------------
  SELECT count(graph_1_c1."id") "_aLias.count"
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
count
293898368


### Get labels, aliases and descriptions for other languages

In [17]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{label_all} -o {output_path}/{output_folder}/labels.{lang}.tsv.gz --match '(n1)-[l:label]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-08 13:40:57 query]: SQL Translation:
---------------------------------------------
  SELECT graph_2_c1."node1", graph_2_c1."label", graph_2_c1."node2", graph_2_c1."id"
     FROM graph_2 AS graph_2_c1
     WHERE graph_2_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_2_c1."node2") = ?)
  PARAS: ['label', 'en']
---------------------------------------------


In [18]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{alias_all} -o {output_path}/{output_folder}/aliases.{lang}.tsv.gz --match '(n1)-[l:alias]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-08 13:46:10 query]: SQL Translation:
---------------------------------------------
  SELECT graph_3_c1."node1", graph_3_c1."label", graph_3_c1."node2", graph_3_c1."id"
     FROM graph_3 AS graph_3_c1
     WHERE graph_3_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_3_c1."node2") = ?)
  PARAS: ['alias', 'en']
---------------------------------------------


In [19]:
for lang in languages:
    cmd = f"kgtk --debug query --graph-cache {os.environ['STORE']} -i {wiki_root_folder}{description_all} -o {output_path}/{output_folder}/descriptions.{lang}.tsv.gz --match '(n1)-[l:description]->(n2)' --where 'n2.kgtk_lqstring_lang_suffix = \"{lang}\"' --return 'n1, l.label, n2, l.id' "
    !{cmd}

[2021-04-08 13:47:21 query]: SQL Translation:
---------------------------------------------
  SELECT graph_4_c1."node1", graph_4_c1."label", graph_4_c1."node2", graph_4_c1."id"
     FROM graph_4 AS graph_4_c1
     WHERE graph_4_c1."label"=?
     AND (kgtk_lqstring_lang_suffix(graph_4_c1."node2") = ?)
  PARAS: ['description', 'en']
---------------------------------------------


### Create the P31 and P279 files

Create the `P31` file

In [20]:
!$kypher -i "$CLAIMS" -o $OUT/derived.P31.tsv.gz \
--match '(n1)-[l:P31]->(n2)' \
--return 'l, n1, l.label, n2' 

[2021-04-08 13:50:11 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P31']
---------------------------------------------


Create the P279 file

In [21]:
!gzcat $OUT/derived.P31.tsv.gz | head | col

/bin/bash: gzcat: command not found



In [22]:
!$kypher -i "$CLAIMS" -o $OUT/derived.P279.tsv.gz \
    --match '(n1)-[l:P279]->(n2)' \
    --return 'l, n1, l.label, n2' 

[2021-04-08 13:56:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P279']
---------------------------------------------


### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279

First compute the roots

In [23]:
!$kypher -i $OUT/derived.P279.tsv.gz -o $TEMP/P279.n1.tsv.gz \
--match '(n1)-[l]->()' \
--return 'n1 as id' 

[2021-04-08 13:59:49 sqlstore]: DROP graph data table graph_5 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279.tsv.gz
[2021-04-08 13:59:52 sqlstore]: IMPORT graph directly into table graph_9 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279.tsv.gz ...
[2021-04-08 13:59:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_9_c1."node1" "_aLias.id"
     FROM graph_9 AS graph_9_c1
  PARAS: []
---------------------------------------------


In [24]:
!$kypher -i $OUT/derived.P31.tsv.gz -o $TEMP/P31.n2.tsv.gz \
--match '()-[l]->(n2)' \
--return 'n2 as id' 

[2021-04-08 14:01:22 sqlstore]: DROP graph data table graph_6 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P31.tsv.gz
[2021-04-08 14:01:43 sqlstore]: IMPORT graph directly into table graph_10 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P31.tsv.gz ...
[2021-04-08 14:02:29 query]: SQL Translation:
---------------------------------------------
  SELECT graph_10_c1."node2" "_aLias.id"
     FROM graph_10 AS graph_10_c1
  PARAS: []
---------------------------------------------


In [25]:
!$kgtk cat --mode NONE -i $TEMP/P31.n2.tsv.gz $TEMP/P279.n1.tsv.gz \
| gzip > $TEMP/P279.roots.1.tsv.gz

In [26]:
!$kgtk sort2 --mode NONE --column id -i $TEMP/P279.roots.1.tsv.gz \
| gzip > $TEMP/P279.roots.2.tsv.gz

We have lots of duplicates

In [27]:
!zcat < $TEMP/P279.roots.2.tsv.gz | head

id
Q100000030
Q1000017
Q1000032
Q1000032
Q1000039
Q100004761
Q100004761
Q100004761
Q1000064

gzip: stdout: Broken pipe


In [28]:
!$kgtk compact -i $TEMP/P279.roots.2.tsv.gz --mode NONE \
    --presorted \
    --columns id \
    -o $TEMP/P279.roots.tsv

Now we can invoke the reachable-nodes command

In [29]:
!$kgtk reachable-nodes \
    --rootfile $TEMP/P279.roots.tsv \
    --selflink \
    -i $OUT/derived.P279.tsv.gz \
| gzip > $TEMP/P279.reachable.tsv.gz

In [30]:
!zcat < $TEMP/P279.reachable.tsv.gz | head | col

node1	label	node2
Q100000030	reachable	Q100000030
Q100000030	reachable	Q14748
Q100000030	reachable	Q14745
Q100000030	reachable	Q1357761
Q100000030	reachable	Q223557

gzip: Q100000030	reachable	Q4406616
Q100000030	reachable	Q488383
stdout: Broken pipe
Q100000030	reachable	Q35120
Q100000030	reachable	Q2424752


The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [31]:
!$kypher -i $TEMP/P279.reachable.tsv.gz -o $TEMP/P279star.1.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "P279star" as label, n2 as node2' 

[2021-04-08 14:47:20 sqlstore]: IMPORT graph directly into table graph_11 from /nas/home/hrathod/output/for_novartis/novartis/temp.useful_files/P279.reachable.tsv.gz ...
[2021-04-08 14:48:36 query]: SQL Translation:
---------------------------------------------
  SELECT graph_11_c1."node1", ? "_aLias.label", graph_11_c1."node2" "_aLias.node2"
     FROM graph_11 AS graph_11_c1
  PARAS: ['P279star']
---------------------------------------------


Now we can concatenate these files to produce the final output

In [32]:
!$kgtk sort2 -i $TEMP/P279star.1.tsv.gz -o $TEMP/P279star.2.tsv.gz

Make sure there are no duplicates

In [33]:
!$kgtk compact --presorted -i $TEMP/P279star.2.tsv.gz -o $TEMP/P279star.3.tsv.gz

Add ids

In [34]:
!$kgtk add-id --id-style node1-label-node2-num -i $TEMP/P279star.3.tsv.gz -o $OUT/derived.P279star.tsv.gz

In [35]:
!zcat < $OUT/derived.P279star.tsv.gz | head | col


gzip: node1	label	node2	id
Q100000030	P279star	Q100000030	Q100000030-P279star-Q100000030-0000
stdout: Broken pipe
Q100000030	P279star	Q1357761	Q100000030-P279star-Q1357761-0000
Q100000030	P279star	Q14745	Q100000030-P279star-Q14745-0000
Q100000030	P279star	Q14748	Q100000030-P279star-Q14748-0000
Q100000030	P279star	Q15401930	Q100000030-P279star-Q15401930-0000
Q100000030	P279star	Q15621286	Q100000030-P279star-Q15621286-0000
Q100000030	P279star	Q16686448	Q100000030-P279star-Q16686448-0000
Q100000030	P279star	Q17537576	Q100000030-P279star-Q17537576-0000
Q100000030	P279star	Q223557 Q100000030-P279star-Q223557-0000


This is how we would do the typical `?item P31/P279* ?class` in Kypher. 
The example shows how to get all the counts of instances of subclasses of city (Q515).

In [36]:
!$kypher -i $OUT/derived.P31.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q515), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, count(c) as count, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'count(c) desc, c, n1' \
--limit 10 \
| col

[2021-04-08 16:06:57 sqlstore]: IMPORT graph directly into table graph_12 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279star.tsv.gz ...
[2021-04-08 16:09:31 sqlstore]: IMPORT graph directly into table graph_13 from /nas/home/hrathod/output/for_novartis/novartis/parts/labels.en.tsv.gz ...
[2021-04-08 16:10:35 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node2" "_aLias.class", count(graph_10_c1."node2") "_aLias.count", graph_13_c4."node2" "_aLias.class name", graph_13_c3."node1" "_aLias.instance", graph_13_c3."node2" "_aLias.label"
     FROM graph_10 AS graph_10_c1, graph_12 AS graph_12_c2, graph_13 AS graph_13_c3, graph_13 AS graph_13_c4
     WHERE graph_10_c1."label"=?
     AND graph_12_c2."node2"=?
     AND graph_13_c3."label"=?
     AND graph_13_c4."label"=?
     AND graph_10_c1."node1"=graph_13_c3."node1"
     AND graph_10_c1."node2"=graph_12_c2."node1"
     AND graph_12_c2."node1"=graph_13_c4."n

Illustrate that it is indeed `P279*`

In [37]:
!$kypher -i $OUT/derived.P31.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" \
--match 'P31: (n1)-[:P31]->(c), P279star: (c)-[]->(:Q63440326), label: (n1)-[:label]->(label), label: (c)-[:label]->(c_label)' \
--return 'distinct c as class, c_label as `class name`, n1 as instance, label as `label`' \
--order-by 'c, n1' \
--limit 10 \
| col 

[2021-04-08 16:29:02 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_13_c4."node1" "_aLias.class", graph_13_c4."node2" "_aLias.class name", graph_10_c1."node1" "_aLias.instance", graph_13_c3."node2" "_aLias.label"
     FROM graph_10 AS graph_10_c1, graph_12 AS graph_12_c2, graph_13 AS graph_13_c3, graph_13 AS graph_13_c4
     WHERE graph_10_c1."label"=?
     AND graph_12_c2."node2"=?
     AND graph_13_c3."label"=?
     AND graph_13_c4."label"=?
     AND graph_10_c1."node1"=graph_13_c3."node1"
     AND graph_10_c1."node2"=graph_12_c2."node1"
     AND graph_12_c2."node1"=graph_13_c4."node1"
     ORDER BY graph_13_c4."node1" ASC, graph_10_c1."node1" ASC
     LIMIT ?
  PARAS: ['P31', 'Q63440326', 'label', 'label', 10]
---------------------------------------------
class	class name	instance	label


Test that `P279star` is indeed star

In [38]:
!$kypher -i $OUT/derived.P279star.tsv.gz \
--match '(n1:Q44)-[:P279star]->(n2:Q44)'

[2021-04-08 16:29:05 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_12 AS graph_12_c1
     WHERE graph_12_c1."label"=?
     AND graph_12_c1."node1"=?
     AND graph_12_c1."node2"=?
  PARAS: ['P279star', 'Q44', 'Q44']
---------------------------------------------
[2021-04-08 16:29:05 sqlstore]: CREATE INDEX on table graph_12 column label ...
[2021-04-08 16:29:54 sqlstore]: ANALYZE INDEX on table graph_12 column label ...
node1	label	node2	id
Q44	P279star	Q44	Q44-P279star-Q44-0000


### Create a file to do generalized Is-A queries
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [39]:
!$kgtk cat -i $OUT/derived.P31.tsv.gz $OUT/derived.P279.tsv.gz \
-o $TEMP/isa.1.tsv.gz

In [40]:
!$kypher -i $TEMP/isa.1.tsv.gz -o $OUT/derived.isa.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "isa" as label, n2' 

[2021-04-08 16:41:29 sqlstore]: DROP graph data table graph_7 from /nas/home/hrathod/output/for_novartis/novartis/temp.useful_files/isa.1.tsv.gz
[2021-04-08 16:41:49 sqlstore]: IMPORT graph directly into table graph_14 from /nas/home/hrathod/output/for_novartis/novartis/temp.useful_files/isa.1.tsv.gz ...
[2021-04-08 16:43:06 query]: SQL Translation:
---------------------------------------------
  SELECT graph_14_c1."node1", ? "_aLias.label", graph_14_c1."node2"
     FROM graph_14 AS graph_14_c1
  PARAS: ['isa']
---------------------------------------------


Example of how to use the `isa` relation

In [41]:
!$kypher -i $OUT/derived.isa.tsv.gz -i $OUT/derived.P279star.tsv.gz -i "$LABELS" -o - \
--match 'isa: (n1)-[l:isa]->(c), P279star: (c)-[]->(:Q44), label: (n1)-[:label]->(label)' \
--return 'distinct n1, l.label, "Q44" as node2, label as n1_label' \
--limit 10 \
| col

[2021-04-08 16:54:44 sqlstore]: DROP graph data table graph_8 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.isa.tsv.gz
[2021-04-08 16:54:53 sqlstore]: IMPORT graph directly into table graph_15 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.isa.tsv.gz ...
[2021-04-08 16:55:40 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_15_c1."node1", graph_15_c1."label", ? "_aLias.node2", graph_13_c3."node2" "_aLias.n1_label"
     FROM graph_12 AS graph_12_c2, graph_13 AS graph_13_c3, graph_15 AS graph_15_c1
     WHERE graph_12_c2."node2"=?
     AND graph_13_c3."label"=?
     AND graph_15_c1."label"=?
     AND graph_12_c2."node1"=graph_15_c1."node2"
     AND graph_13_c3."node1"=graph_15_c1."node1"
     LIMIT ?
  PARAS: ['Q44', 'Q44', 'label', 'isa', 10]
---------------------------------------------
[2021-04-08 16:55:40 sqlstore]: CREATE INDEX on table graph_15 column node2 ...
[2021-04-08 16:58:58 sql

### Create files with `isa/P279* and P31/P279*` 
This file is useful to find all nodes that are below a q-node via P279 or isa.

> These files are very large and take many hours to compute

In [9]:
os.environ['P279STAR'] = f"{os.environ['OUT']}/derived.P279star.tsv.gz"
os.environ['ISA'] = f"{os.environ['OUT']}/derived.isa.tsv.gz"

In [10]:
!ls -l $CLAIMS
!ls -l $P279STAR
!ls -l $ISA

-rw-r--r-- 1 hrathod div22 6192684492 Apr  8 04:06 /nas/home/hrathod/output/for_novartis/novartis/parts/claims.tsv.gz
-rw-r--r-- 1 hrathod div22 500270514 Apr  8 16:06 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279star.tsv.gz
-rw-r--r-- 1 hrathod div22 117261297 Apr  8 16:54 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.isa.tsv.gz


In [44]:
!$kypher -i "$P279STAR" -i "$ISA"  \
--match '\
  isa: (n1)-[]->(n2), \
  P279star: (n2)-[]->(n3)' \
--return 'distinct n1 as node1, "isa_star" as label, n3 as node2' \
-o "$TEMP"/derived.isastar_1.tsv.gz

[2021-04-08 17:00:00 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_15_c1."node1" "_aLias.node1", ? "_aLias.label", graph_12_c2."node2" "_aLias.node2"
     FROM graph_12 AS graph_12_c2, graph_15 AS graph_15_c1
     WHERE graph_12_c2."node1"=graph_15_c1."node2"
  PARAS: ['isa_star']
---------------------------------------------


Now add ids and sort it

In [45]:
!$kgtk add-id --id-style wikidata -i "$TEMP"/derived.isastar_1.tsv.gz \
/ sort2 -o "$OUT"/derived.isastar.tsv.gz

It is very big

In [46]:
!zcat < "$OUT"/derived.isastar.tsv.gz | wc

1205919877 4823679508 66737179355


Also calculate the same file by for P31/P279*

In [20]:
!echo $P279STAR

/nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279star.tsv.gz


In [21]:
!$kypher -i "$CLAIMS" -i "$P279STAR" \
--match '\
  claims: (n1)-[:P31]->(n2), \
  P279star: (n2)-[]->(n3)' \
--return 'distinct n1 as node1, "P31P279star" as label, n3 as node2' \
-o "$TEMP"/derived.P31P279star.gz

[2021-04-09 09:43:32 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", ? "_aLias.label", graph_12_c2."node2" "_aLias.node2"
     FROM graph_1 AS graph_1_c1, graph_12 AS graph_12_c2
     WHERE graph_1_c1."label"=?
     AND graph_12_c2."node1"=graph_1_c1."node2"
  PARAS: ['P31P279star', 'P31']
---------------------------------------------


Add ids and sort it

In [22]:
!$kgtk add-id --id-style wikidata -i "$TEMP"/derived.P31P279star.gz \
/ sort2 -o "$OUT"/derived.P31P279star.tsv.gz

It is also very big

In [23]:
!zcat < "$OUT"/derived.P31P279star.tsv.gz | wc

1189725419 4758901676 72980964319


## Compute pagerank

Now compute pagerank. These commands will exceed 16GB memory for graphs containing over 25 million nodes.

In [24]:
if compute_pagerank:
    !$kgtk graph-statistics -i "$ITEMS" -o $OUT/metadata.pagerank.directed.tsv.gz \
    --page-rank-property directed_pagerank \
    --pagerank --statistics-only \
    --log $TEMP/metadata.pagerank.directed.summary.txt 

Matplotlib is building the font cache; this may take a moment.


In [25]:
if compute_pagerank:
    !cat $TEMP/metadata.pagerank.directed.summary.txt

graph loaded! It has 33937298 nodes and 110441933 edges

###Top relations:
P31	33987360
P6259	7999222
P59	7350483
P17	5023882
P684	4314512
P361	3219107
P703	3061100
P279	2926455
P171	2901200
P105	2892790

###PageRank
Max pageranks
3081	Q4167836	0.035786
81738	Q1264450	0.011216
341462	Q35252665	0.009717
1906	Q11266439	0.008379
587934	Q17442446	0.008315


In [26]:
if compute_pagerank:
    !$kgtk graph-statistics -i "$ITEMS" -o $OUT/metadata.pagerank.undirected.tsv.gz \
    --page-rank-property undirected_pagerank \
    --pagerank --statistics-only --undirected \
    --log $TEMP/metadata.pagerank.undirected.summary.txt 

In [27]:
if compute_pagerank:
    !cat $TEMP/metadata.pagerank.undirected.summary.txt 

graph loaded! It has 33937298 nodes and 110441933 edges

###Top relations:
P31	33987360
P6259	7999222
P59	7350483
P17	5023882
P684	4314512
P361	3219107
P703	3061100
P279	2926455
P171	2901200
P105	2892790

###PageRank
Max pageranks
3081	Q4167836	0.039628
81738	Q1264450	0.036635
7291	Q4167410	0.015681
13948	Q523	0.015324
6740	Q11173	0.012707


## Compute Degrees

Kypher can compute the out degree by counting the node2s for each node1

In [28]:
!$kypher -i "$CLAIMS" -o $TEMP/metadata.out_degree.tsv.gz \
--match '(n1)-[l]->()' \
--return 'distinct n1 as node1, count(distinct l) as node2, "out_degree" as label' 

[2021-04-10 01:26:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node1" "_aLias.node1", count(DISTINCT graph_1_c1."id") "_aLias.node2", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     GROUP BY "_aLias.node1"
  PARAS: ['out_degree']
---------------------------------------------


In [29]:
!$kgtk add-id --id-style wikidata -i $TEMP/metadata.out_degree.tsv.gz \
/ sort2 -o $OUT/metadata.out_degree.tsv.gz

To count the in-degree we only care when the node2 is a wikibase-item

In [30]:
# BUG in kypher, sometimes the following command will not work, as in we'll see multilple rows for a Qnode, which is
# fixable by deleting cache
!$kypher -i "$CLAIMS" -o $TEMP/metadata.in_degree.tsv.gz \
--match '()-[l]->(n2 {`wikidatatype`:"wikibase-item"})' \
--return 'distinct n2 as node1, count(distinct l) as node2, "in_degree" as label' \
--order-by 'n2'

[2021-04-10 01:58:42 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_1_c1."node2" "_aLias.node1", count(DISTINCT graph_1_c1."id") "_aLias.node2", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."node2;wikidatatype"=?
     GROUP BY "_aLias.node1"
     ORDER BY graph_1_c1."node2" ASC
  PARAS: ['in_degree', 'wikibase-item']
---------------------------------------------


In [31]:
# rename columns before adding ids, KGTK will complain
# df = pd.read_csv('{}/metadata.in_degree.tsv.gz'.format(os.environ['TEMP']), sep='\t')
# df = df.rename(columns={"node2": "node1"})
# df = df.rename(columns={'count(DISTINCT graph_1_c1."id")': "node2"})
# df.to_csv('{}/metadata.in_degree.1.tsv.gz'.format(os.environ['TEMP']), sep='\t', index=False)

In [32]:
!$kgtk add-id --id-style wikidata -i $TEMP/metadata.in_degree.tsv.gz \
/ sort2 -o $OUT/metadata.in_degree.tsv.gz

In [33]:
!zcat < $OUT/metadata.in_degree.tsv.gz | head | col


gzip: node1	node2	label	id
stdout: Broken pipe
Q1	74	in_degree	Q1-in_degree-eb624d
Q1000	2124	in_degree	Q1000-in_degree-70e8d5
Q10000002	1	in_degree	Q10000002-in_degree-6b86b2
Q100000040	1	in_degree	Q100000040-in_degree-6b86b2
Q100000143	12	in_degree	Q100000143-in_degree-6b51d4
Q10000019	1	in_degree	Q10000019-in_degree-6b86b2
Q100000259	1	in_degree	Q100000259-in_degree-6b86b2
Q100000265	2	in_degree	Q100000265-in_degree-d4735e
Q100000271	1	in_degree	Q100000271-in_degree-6b86b2


Calculate the distribution so we can make a nice chart

In [34]:
!$kypher -i $OUT/metadata.in_degree.tsv.gz -o $OUT/statistics.in_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as in_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

[2021-04-10 07:28:48 sqlstore]: IMPORT graph directly into table graph_16 from /nas/home/hrathod/output/for_novartis/novartis/useful_files/metadata.in_degree.tsv.gz ...
[2021-04-10 07:29:08 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_16_c1."node2" "_aLias.in_degree", count(DISTINCT graph_16_c1."node1") "_aLias.count", ? "_aLias.label"
     FROM graph_16 AS graph_16_c1
     GROUP BY "_aLias.in_degree"
     ORDER BY CAST(graph_16_c1."node2" AS integer) ASC
  PARAS: ['count']
---------------------------------------------


In [35]:
!head $OUT/statistics.in_degree.distribution.tsv | col

in_degree	count	label
1	3834601 count
2	797912	count
3	379657	count
4	143176	count
5	93515	count
6	64805	count
7	57950	count
8	35678	count
9	28832	count


In [11]:
!$kypher -i $OUT/metadata.out_degree.tsv.gz -o $OUT/statistics.out_degree.distribution.tsv \
--match '(n1)-[]->(n2)' \
--return 'distinct n2 as out_degree, count(distinct n1) as count, "count" as label' \
--order-by 'cast(n2, integer)' 

[2021-04-10 13:11:07 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_17_c1."node2" "_aLias.out_degree", count(DISTINCT graph_17_c1."node1") "_aLias.count", ? "_aLias.label"
     FROM graph_17 AS graph_17_c1
     GROUP BY "_aLias.out_degree"
     ORDER BY CAST(graph_17_c1."node2" AS integer) ASC
  PARAS: ['count']
---------------------------------------------


Draw some charts

In [12]:
data = pd.read_csv(
    os.environ["OUT"] + "/statistics.in_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("in_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["in_degree", "count"],
).interactive().properties(title="Distribution of In Degree")

alt.Chart(...)

In [13]:
data = pd.read_csv(
    os.environ["OUT"] + "/statistics.out_degree.distribution.tsv", sep="\t"
)

alt.Chart(data).mark_circle(size=60).encode(
    x=alt.X("out_degree", scale=alt.Scale(type="log")),
    y=alt.Y("count", scale=alt.Scale(type="log"), title="count of nodes"),
    tooltip=["out_degree", "count"],
).interactive().properties(title="Distribution of Out Degree")

alt.Chart(...)

## Summary of results

In [14]:
!ls -lh $OUT/*

-rw-r--r-- 1 hrathod div22  86M Apr  8 13:47 /nas/home/hrathod/output/for_novartis/novartis/useful_files/aliases.en.tsv.gz
-rw-r--r-- 1 hrathod div22 7.3G Apr  9 01:57 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.isastar.tsv.gz
-rw-r--r-- 1 hrathod div22 112M Apr  8 16:54 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.isa.tsv.gz
-rw-r--r-- 1 hrathod div22 478M Apr  8 16:06 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279star.tsv.gz
-rw-r--r-- 1 hrathod div22  36M Apr  8 13:59 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P279.tsv.gz
-rw-r--r-- 1 hrathod div22 7.3G Apr  9 19:37 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P31P279star.tsv.gz
-rw-r--r-- 1 hrathod div22 410M Apr  8 13:56 /nas/home/hrathod/output/for_novartis/novartis/useful_files/derived.P31.tsv.gz
-rw-r--r-- 1 hrathod div22 167M Apr  8 13:50 /nas/home/hrathod/output/for_novartis/novartis/useful_files/descriptio

Highest page rank

In [17]:
!echo $OUT/metadata.pagerank.undirected.tsv.gz

/nas/home/hrathod/output/for_novartis/novartis/useful_files/metadata.pagerank.undirected.tsv.gz


In [21]:

if compute_pagerank:
    !$kypher -i $OUT/metadata.pagerank.undirected.tsv.gz -i "$LABELS" -o - \
    --match 'pagerank: (n1)-[:undirected_pagerank]->(page_rank), label: (n1)-[:label]->(label)' \
    --return 'distinct n1, label as label, page_rank as `undirected page rank' \
    --order-by 'cast(page_rank, float) desc' \
    --limit 10 \
    | col

Traceback (most recent call last):
  File "/nas/home/hrathod/anaconda3/envs/c_k_g/lib/python3.7/site-packages/kgtk/cli/query.py", line 174, in run
    index=options.get('index'))
  File "/nas/home/hrathod/anaconda3/envs/c_k_g/lib/python3.7/site-packages/kgtk/kypher/query.py", line 177, in __init__
    self.query = parser.intern(query)
  File "/nas/home/hrathod/anaconda3/envs/c_k_g/lib/python3.7/site-packages/kgtk/kypher/parser.py", line 890, in intern
    return KypherQuery(query_string)
  File "/nas/home/hrathod/anaconda3/envs/c_k_g/lib/python3.7/site-packages/kgtk/kypher/parser.py", line 811, in __init__
    self.query = intern_ast(self, self.parse.Kypher())
  File "/nas/home/hrathod/anaconda3/envs/c_k_g/lib/python3.7/site-packages/parsley.py", line 98, in invokeRule
    raise err
ometa.runtime.ParseError: 
 MATCH pagerank: (n1)-[:undirected_pagerank]->(page_rank), label: (n1)-[:label]->(label) RETURN distinct n1, label as label, page_rank as `undirected page rank ORDER BY cast(page_

In [22]:
!ls -l $LABELS

-rw-r--r-- 1 hrathod div22 390663548 Apr  8 04:30 /nas/home/hrathod/output/for_novartis/novartis/parts/labels.en.tsv.gz
